<a href="https://colab.research.google.com/github/mleyvaz/IO/blob/main/Copia_de_vrp_capacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# vrp_capacity

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/master/examples/notebook/constraint_solver/vrp_capacity.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/master/ortools/constraint_solver/samples/vrp_capacity.py"><img src="https://raw.githubusercontent.com/google/or-tools/master/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
!pip install ortools

     |████████████████████████████████| 14.0MB 341kB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 1.0MB 39.2MB/s 
ERROR: tensorflow-metadata 0.27.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.
  Found existing installation: absl-py 0.10.0
    Uninstalling absl-py-0.10.0:
      Successfully uninstalled absl-py-0.10.0
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [2]:
# Copyright 2010-2018 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# [START program]
"""Capacited Vehicles Routing Problem (CVRP)."""

# [START import]
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]


# [START data_model]
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [[ 0.        ,  0.58960333,  4.2383701 ,  8.02774556,  8.32925408,
         9.25273012,  9.88868557, 11.07272647, 11.4304199 , 12.33900852,
        12.90465359],
       [ 0.58960333,  0.        ,  4.72922469,  8.57632794,  8.88362681,
         9.81418988, 10.44417266, 11.618726  , 11.97434772, 12.87867052,
        13.44133548],
       [ 4.2383701 ,  4.72922469,  0.        ,  4.03095992,  4.38251463,
         5.36284619,  5.91853717,  7.00939235,  7.35061367,  8.22904887,
         8.7771992 ],
       [ 8.02774556,  8.57632794,  4.03095992,  0.        ,  0.38439214,
         1.36036158,  1.88952884,  3.04602758,  3.40684142,  4.32580776,
         4.90114321],
       [ 8.32925408,  8.88362681,  4.38251463,  0.38439214,  0.        ,
         0.98641947,  1.56095371,  2.76716649,  3.1345218 ,  4.06314969,
         4.64447147],
       [ 9.25273012,  9.81418988,  5.36284619,  1.36036158,  0.98641947,
         0.        ,  0.70537981,  1.98028231,  2.35127254,  3.28105142,
         3.86716797],
       [ 9.88868557, 10.44417266,  5.91853717,  1.88952884,  1.56095371,
         0.70537981,  0.        ,  1.27508404,  1.64639425,  2.57736606,
         3.16362734],
       [11.07272647, 11.618726  ,  7.00939235,  3.04602758,  2.76716649,
         1.98028231,  1.27508404,  0.        ,  0.37210671,  1.30542694,
         1.89138359],
       [11.4304199 , 11.97434772,  7.35061367,  3.40684142,  3.1345218 ,
         2.35127254,  1.64639425,  0.37210671,  0.        ,  0.93347493,
         1.51931021],
       [12.33900852, 12.87867052,  8.22904887,  4.32580776,  4.06314969,
         3.28105142,  2.57736606,  1.30542694,  0.93347493,  0.        ,
         0.58627987],
       [12.90465359, 13.44133548,  8.7771992 ,  4.90114321,  4.64447147,
         3.86716797,  3.16362734,  1.89138359,  1.51931021,  0.58627987,
         0.        ]]
    # [START demands_capacities]
    data['demands'] = [15, 0.3, 0.8, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 15]
    data['vehicle_capacities'] = [15, 15]
    # [END demands_capacities]
    data['num_vehicles'] = 2
    data['depot'] = 1
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    # [END solution_printer]


"""Solve the CVRP problem."""
# Instantiate the data problem.
# [START data]
data = create_data_model()
# [END data]

# Create the routing index manager.
# [START index_manager]
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
# [END index_manager]

# Create Routing Model.
# [START routing_model]
routing = pywrapcp.RoutingModel(manager)

# [END routing_model]

# Create and register a transit callback.
# [START transit_callback]
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
# [END transit_callback]

# Define cost of each arc.
# [START arc_cost]
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# [END arc_cost]

# Add Capacity constraint.
# [START capacity_constraint]
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
    demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # null capacity slack
    data['vehicle_capacities'],  # vehicle maximum capacities
    True,  # start cumul to zero
    'Capacity')
# [END capacity_constraint]

# Setting first solution heuristic.
# [START parameters]
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)
# [END parameters]

# Solve the problem.
# [START solve]
solution = routing.SolveWithParameters(search_parameters)
# [END solve]

# Print solution on console.
# [START print_solution]
if solution:
    print_solution(data, manager, routing, solution)
# [END print_solution]



Route for vehicle 0:
 1 Load(0.3) ->  2 Load(1.1) ->  3 Load(1.4000000000000001) ->  6 Load(1.7000000000000002) ->  8 Load(2.0) ->  9 Load(2.3) ->  10 Load(17.3) ->  7 Load(17.6) ->  5 Load(17.900000000000002) ->  4 Load(18.200000000000003) ->  1 Load(18.200000000000003)
Distance of the route: 20m
Load of the route: 18.200000000000003

Route for vehicle 1:
 1 Load(0.3) ->  0 Load(15.3) ->  1 Load(15.3)
Distance of the route: 0m
Load of the route: 15.3

Total distance of all routes: 20m
Total load of all routes: 33.5
